In [1]:
from pathlib import Path

import geopandas as gpd
import folium
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
from bmi_topography import Topography
from ipywidgets import interact
from matplotlib.colors import LogNorm

In [2]:
CONFIG_FILE = "../config.yml"
with open(CONFIG_FILE) as handle:
    config = yaml.safe_load(handle)


data_dir = Path(config['global']['data_dir'])
slr_path = data_dir / config['demo']['slr_file']
damage_path = data_dir / config['demo']['damage_file']
topo_cache_dir = data_dir / config['demo']['topo_cache_subdir']

opentopo_api_key = config['demo']['opentopo_api_key']

In [3]:
slr_data = pd.read_csv(slr_path).pivot(index='City', columns='Year', values='SLR')
# display(slr_data)
damage_data = pd.read_csv(damage_path, index_col='City')
damage_data.columns = slr_data.columns
# display(damage_data)
# print(damage_data.index.symmetric_difference(slr_data.index))

In [4]:
@interact(city=slr_data.index)
def plot(city):
    fig, ax = plt.subplots(figsize=(12, 6))

    slr_data.loc[city].plot(label="Sea level rise (left)", ax=ax)
    ax.set_ylabel("Mean modelled SLR (metres)")
    ax.set_title(city)

    ax2 = damage_data.loc[city].plot(secondary_y=True, ax=ax, label="Damages")
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)
    ax2.set_ylabel("Expected damages under RCP8.5 (million USD (2005))")
    
    location = gpd.tools.geocode(city, provider='nominatim', user_agent='test')
    
    f = folium.Figure(width=12*65, height=6*65)
    m = location.explore(tiles='CartoDB positron', zoom_start=12)
    m.add_to(f)
    display(f)
    
    lonlat = location.at[0, 'geometry'].coords[0]
    topo = Topography(
        dem_type='SRTMGL3',
        south=lonlat[1]-.1,
        north=lonlat[1]+.1,
        west=lonlat[0]-.1,
        east=lonlat[0]+.1,
        output_format='GTiff',
        cache_dir=topo_cache_dir,
        api_key=opentopo_api_key
    )
    topo.fetch()
    data = topo.load()
    fig, ax = plt.subplots(figsize=(12, 6))
    data[0].plot(cmap='Greys', vmin=data[0].min(), vmax=20, ax=ax)
    data[0].plot.contourf(levels=[0, 10], ax=ax, colors=['none', (1, 0, 0, 0.2), 'none'])

interactive(children=(Dropdown(description='city', options=('Abidjan', 'Accra', 'Adelaide', 'Alexandria', 'Alg…